In [ ]:
# ==========================================
# 1. 导入库 (保持 QONNX 修正)
# ==========================================
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import GiveUniqueNodeNames
from finn.core.onnx_exec import execute_onnx
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from qonnx.custom_op.registry import getCustomOp
import numpy as np
import os

# ==========================================
# 2. 配置
# ==========================================
build_dir = "output_lenet" 
test_fpga_part = "xc7z020clg400-1"
target_clk_ns = 10 

print(f"🚀 [Step 1] 初始化...")

# ==========================================
# 3. 加载模型
# ==========================================
model_path = build_dir + "/lenet_folded.onnx"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"❌ 找不到 {model_path}")

model = ModelWrapper(model_path)

print("🔄 [Step 2] 编译仿真环境...")
model = model.transform(GiveUniqueNodeNames())
model = model.transform(PrepareIP(test_fpga_part, target_clk_ns))
model = model.transform(HLSSynthIP())
model = model.transform(SetExecMode("rtlsim"))
model = model.transform(PrepareRTLSim())

# ==========================================
# 4. 配置节点 (已修复判定逻辑)
# ==========================================
print("🔧 [Step 3] 配置波形记录...")
configured_count = 0

for node in model.graph.node:
    # 🌟 修复点：使用 'in' 来匹配，兼容 .rtl 和 .hls 后缀
    if "finn.custom_op.fpgadataflow" in node.domain:
        try:
            node_inst = getCustomOp(node)
            node_inst.set_nodeattr("exec_mode", "rtlsim")
            node_inst.set_nodeattr("rtlsim_trace", "trace.vcd")
            node_inst.set_nodeattr("cycles_rtlsim", 0)
            configured_count += 1
        except Exception as e:
            print(f"   ⚠️ 配置失败 {node.name}: {e}")

print(f"✅ 已配置 {configured_count} 个节点 (这次应该不为0了！)")

if configured_count == 0:
    raise ValueError("❌ 仍然没有找到节点？请检查诊断信息中 Domain 的拼写！")

# ==========================================
# 5. 运行仿真
# ==========================================
print("🎲 [Step 4] 生成数据...")
input_name = model.graph.input[0].name
input_shape = model.get_tensor_shape(input_name)
input_data = np.random.randint(0, 256, size=input_shape).astype(np.float32)

print(f"🚀 [Step 5] 启动仿真 (请耐心等待)...")
execute_onnx(model, {input_name: input_data})

print("\n仿真完成！性能报告：")
print(f"{'='*40}")
time_us_all = 0
cycles_all  = 0
for node in model.graph.node:
    if "finn.custom_op.fpgadataflow" in node.domain:
        try:
            node_inst = getCustomOp(node)
            cycles = node_inst.get_nodeattr("cycles_rtlsim")
            if cycles:
                time_us = (cycles * target_clk_ns) / 1000.0
                time_us_all += time_us
                cycles_all += cycles
                print(f"🔹 {node.name}: {cycles} cycles ({time_us} us)")
        except: pass
print(f"{'='*40}")
print(f"Total Time: {cycles_all} cycles ({time_us_all} us)")

# 自动找波形
print("\n波形文件位置：")
for root, dirs, files in os.walk(build_dir):
    for file in files:
        if file == "trace.vcd" and os.path.getsize(os.path.join(root, file)) > 0:
            print(f"📍 {os.path.join(root, file)}")

In [ ]:
# ==========================================
# 1. 导入库
# ==========================================
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import GiveUniqueNodeNames
from finn.core.onnx_exec import execute_onnx
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.fpgadataflow.insert_fifo import InsertFIFO
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from qonnx.custom_op.registry import getCustomOp
import numpy as np
import os

# ==========================================
# 2. 基础配置
# ==========================================
build_dir = "output_lenet_stitched"
if not os.path.exists(build_dir):
    os.makedirs(build_dir)
    
test_fpga_part = "xc7z020clg400-1"
target_clk_ns = 10 

# 你的“折叠后”模型 (Child Model)
child_model_path = "output_lenet/lenet_folded.onnx"
# 你的“父模型” (之前步骤生成的)
parent_model_path = "output_lenet/lenet_dataflow_parent.onnx"

print(f"🚀 [Step 1] 加载子模型: {child_model_path}")
model = ModelWrapper(child_model_path)

# ==========================================
# 3. 处理子模型 (插入 FIFO/DWC -> Stitching)
# ==========================================
print("🔧 [Step 2] 插入 DWC 和 FIFO 以匹配数据流...")
model = model.transform(InsertDWC())
model = model.transform(InsertFIFO(create_shallow_fifos=True))
model = model.transform(SpecializeLayers(test_fpga_part))
model = model.transform(GiveUniqueNodeNames())

print("⚙️ [Step 3] 生成 IP 并执行 Stitching (HLS Synthesis)...")
model = model.transform(PrepareIP(test_fpga_part, target_clk_ns))
model = model.transform(HLSSynthIP())
model = model.transform(CreateStitchedIP(test_fpga_part, target_clk_ns))

print("🔌 [Step 4] 准备 PyVerilator 仿真文件...")
model = model.transform(PrepareRTLSim())

# 设置子模型的执行模式为 rtlsim
model.set_metadata_prop("exec_mode", "rtlsim")
model.set_metadata_prop("rtlsim_trace", "trace_stitched.vcd")

# 保存处理好的子模型
stitched_child_path = build_dir + "/lenet_stitched_child.onnx"
model.save(stitched_child_path)
print(f"💾 子模型已保存: {stitched_child_path}")


In [ ]:
# 设置超时阈值为 1,000,000 周期 (足够应对 20k 的瓶颈)
os.environ["LIVENESS_THRESHOLD"] = "1000000"
print("🚀 [Step 5] 启动 Stitched IP 仿真 (Direct Child Execution)...")

# 1. 确保子模型被标记为 rtlsim (之前已经做过，再次确认)
model.set_metadata_prop("exec_mode", "rtlsim")
model.set_metadata_prop("rtlsim_trace", "top_level_trace.vcd")

# 2. 生成符合子模型输入的随机数据
# 注意：子模型的输入是 stream 接口前的格式，通常和 Parent 输入一致
input_name = model.graph.input[0].name
input_shape = model.get_tensor_shape(input_name)
input_data = np.random.randint(0, 256, size=input_shape).astype(np.float32)

input_dict = {input_name: input_data}

# 3. 执行仿真
# 这次我们直接传入 'model' (即 child model)，而不是 'parent_model'
output_dict = execute_onnx(model, input_dict)

print("✅ Stitched IP 仿真完成！")

# ==========================================
# 6. 查找并验证波形
# ==========================================
print(f"\n📈 正在搜索波形文件...")

# 这里的逻辑是：Stitched IP 的 trace 文件通常会生成在
# 1. output_lenet_stitched 目录下
# 2. 或者在 vivado_stitch_proj 目录下

target_file = "top_level_trace.vcd"
found_path = None

# 方法 A: 检查 build_dir
if os.path.exists(os.path.join(build_dir, target_file)):
    found_path = os.path.join(build_dir, target_file)

# 方法 B: 全局搜索 (为了保险)
if not found_path:
    for root, dirs, files in os.walk(build_dir):
        if target_file in files:
            found_path = os.path.join(root, target_file)
            break

if found_path:
    size_mb = os.path.getsize(found_path) / (1024 * 1024)
    print(f"🎉 成功找到顶层波形文件！")
    print(f"📍 路径: {found_path}")
    print(f"📦 大小: {size_mb:.2f} MB")
    print(f"\n💡 使用 GTKWave 打开命令: gtkwave {found_path}")
else:
    print("⚠️ 仿真看似成功，但在当前目录下未找到 .vcd 文件。")
    print("它可能位于系统的临时目录 (/tmp/finn_dev_...) 中。")
    # 尝试从 metadata 中读取真实路径
    try:
        trace_loc = model.get_metadata_prop("rtlsim_trace_fullpath")
        if trace_loc and os.path.exists(trace_loc):
            print(f"📍 从 Metadata 找到路径: {trace_loc}")
    except:
        pass

In [ ]:
# ==========================================
# 9. [批量测试] 运行 100 次全网验证
# ==========================================
import numpy as np
import tqdm  # 用于显示进度条
from finn.core.onnx_exec import execute_onnx

print(f"开始批量测试 (100 次随机验证)...")

# 计数器
total_tests = 100
passed_count = 0

# 确保模型 exec_mode 设置正确
golden_model.set_metadata_prop("exec_mode", "") 
model.set_metadata_prop("exec_mode", "rtlsim")

# 获取输入输出节点名
input_name_rtl = model.graph.input[0].name
output_name_rtl = model.graph.output[0].name
input_name_golden = golden_model.graph.input[0].name
output_name_golden = golden_model.graph.output[0].name
input_shape = model.get_tensor_shape(input_name_rtl)

# 使用 tqdm 显示进度条
for i in tqdm.tqdm(range(total_tests)):
    try:
        # 1. 生成随机输入 (UINT8, 0-255, NHWC)
        # 这里的 input_shape 通常是 (1, 32, 32, 1)
        rand_input = np.random.randint(0, 256, size=input_shape).astype(np.float32)
        
        # 2. 准备 RTL 输入字典
        input_dict_rtl = {input_name_rtl: rand_input}
        
        # 3. 准备 Golden 输入 (归一化 + 转置 NHWC -> NCHW)
        input_data_golden = rand_input.astype(np.float32) / 255.0
        if input_data_golden.shape == (1, 32, 32, 1):
             input_data_golden = input_data_golden.transpose(0, 3, 1, 2)
        
        # 4. 执行 RTL 仿真
        ret_rtl = execute_onnx(model, input_dict_rtl)
        out_rtl = ret_rtl[output_name_rtl]
        
        # 5. 执行 Golden 仿真
        ret_golden = execute_onnx(golden_model, {input_name_golden: input_data_golden})
        out_golden = ret_golden[output_name_golden]
        
        # 6. 对比结果 (Argmax)
        # 软件输出通常是 Logits，硬件输出可能是 Class ID 或 One-hot
        # 我们统一转成 Class ID 进行对比
        
        # Golden (Logits) -> Class ID
        pred_golden = np.argmax(out_golden, axis=1).flatten()[0]
        
        # RTL -> Class ID
        pred_rtl = out_rtl.flatten()
        # 如果 RTL 输出是 One-hot (例如 shape=(1, 10))
        if out_rtl.shape[1] > 1:
            pred_rtl = np.argmax(out_rtl, axis=1).flatten()
        pred_rtl = pred_rtl[0]
        
        # 判定
        if pred_rtl == pred_golden:
            passed_count += 1
        else:
            # 只有出错时才打印详细信息，避免刷屏
            print(f"\n Test {i+1} Failed: RTL pred={pred_rtl}, Golden pred={pred_golden}")
            print(f"Golden pred vector:{out_golden}")
            
    except Exception as e:
        print(f"\n⚠️ Test {i+1} Error: {e}")

# ==========================================
# 最终报告
# ==========================================
accuracy = (passed_count / total_tests) * 100
print("\n" + "="*40)
print(f"测试摘要 (Total: {total_tests})")
print(f"通过次数: {passed_count}")
print(f"失败次数: {total_tests - passed_count}")
print(f"最终匹配率: {accuracy:.2f}%")
print("="*40)

if passed_count == total_tests:
    print("PASS!所有 100 次随机测试全部通过")
else:
    print("存在不一致!请检查由于量化精度处于边界导致的偶尔跳变。")